<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação de pacotes

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


#Importar pacotes


In [3]:
import pandas as pd
import string
import time
import csv
import random
from ast import literal_eval
from textblob import TextBlob
from google.colab import drive

#import para mitigar erro au acessar aux.sentences
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Baixar dataset

In [ ]:
#baixa o arquivo do dataset para o ambiente colab
!wget "http://tiagodemelo.info/datasets/dataset-v2.dat"



--2020-11-20 15:12:29--  http://tiagodemelo.info/datasets/dataset-v2.dat
Resolving tiagodemelo.info (tiagodemelo.info)... 108.167.188.189
Connecting to tiagodemelo.info (tiagodemelo.info)|108.167.188.189|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89773865 (86M)
Saving to: ‘dataset-v2.dat’

dataset-v2.dat      100%[===================>]  85.61M  17.4MB/s    in 4.9s    

2020-11-20 15:12:34 (17.4 MB/s) - ‘dataset-v2.dat’ saved [89773865/89773865]



In [ ]:
# ler o arquivo com dos dados do dataset baixado
file = open("./dataset-v2.dat",'r')

# sepera em linhas os dados
rows = file.readlines()

# fecha o arquivo
file.close()

# criar uma lista para salvar os dicionarios
dictionaries = []

for row in rows:
  #transforma literal em dicionario
  saveDic = literal_eval(row)

  #adiciona atributos de subdicionarios no dicionario mais externo
  saveDic.update(saveDic['itemReviewed'].items())
  saveDic.update(saveDic['itemReviewed']['address'].items())

  # #delete subdicionarios
  del saveDic['address']
  del saveDic['itemReviewed']

  #adiciona na lista de dicionarios
  dictionaries.append(saveDic)




In [ ]:
# transforma a lista de dicionario em dataframe
data = pd.DataFrame(dictionaries)

data.head()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,ratingValue,name,website,email,phoneNumber,averageRating,streetAddress,addressLocality,addressRegion,postalCode,country
0,Jantar,692856288,2344932,larinhavercosa222,"Comida maravilhosa, lugar do verdadeiro chicle...",pt,True,"São Carlos, SP",26/07/2019,07/2019,5,Imperador dos Camarões,http://www.imperadordoscamaroes.com.br,contato@imperadordoscamaroes.com.br,+55 82 3231-4134,4.5,Avenida Doutor Antônio Gouveia,,Alagoas,57030-170,Brasil
1,Uma boa experiência.,229208487,3720906,Pano P,"Comida boa, drinks legais e um ambiente bem ac...",pt,True,"Manaus, AM",16/09/2014,None,4,Guacamole Natal,http://www.facebook.com/GuacamoleNatal,guacamolenatal@gmail.com,+55 84 3219-3646,4.5,Avenida Praia de Ponta Negra 9068 Ponta Negra,,Rio Grande do Norte,59092-100,Brasil
2,Otimo self service,679002332,2248105,Magma100,"Excelente pra ir com a família! Ótima Comida, ...",pt,True,"Goiânia, GO",04/06/2019,07/2018,4,Chica Pitanga,http://www.chicapitanga.com.br,chicapitanga@chicapitanga.com.br,+55 81 3334-8260,4.5,Rua Petrolina 19,Recife,Pernambuco,51021-250,Brasil
3,O sorvete mais tradicional do estado,355671519,8168302,Yurie M,Tomo sorvete na Visabor faz mais de 30 anos. S...,pt,False,"Aracaju, SE",15/03/2016,01/2016,5,Sorvetes Vi Sabor,None,None,None,4.0,Avenida Francisco Porto 59 Grageru,Aracaju,Sergipe,,Brasil
4,Melhor hamburger!,268741921,5395141,Debora_C85,O melhor hamburger que já comi! Excelente aten...,pt,False,"Rio de Janeiro, RJ",28/04/2015,04/2015,5,Z Deli Sandwich Shop,http://www.facebook.com/zdelisandwichshop/,None,551130830021,4.5,Rua Haddock Lobo 1386 Jardins,São Paulo,Estado de São Paulo,01414-002,Brasil


#Tratamento de Comentários Vazios

In [ ]:
#seleciona tuplas vazias
data[pd.isnull(data.reviewBody)]

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,ratingValue,name,website,email,phoneNumber,averageRating,streetAddress,addressLocality,addressRegion,postalCode,country
8528,Vale a pena conhecer!,224445539,NaN,NaN,NaN,pt,False,"Recife, PE",25/08/2014,12/2013,4,Outback Steakhouse - Shopping RioMar Recife,https://www.outback.com.br/restaurantes/shoppi...,faleconosco@outback.com.br,+55 81 3035-0930,4.5,"Av República do Líbano, 256 Shopping RioMar Re...",Recife,Pernambuco,51110-160,Brasil


In [ ]:
data.reviewBody.fillna('VAZIO')

data.reviewBody[8].encode('utf-16', 'surrogatepass').decode('utf-16')

'Ambiente agradável, preço totalmente acessível, e adoro o cardápio de lá! Sem falar no atendimento maravilhoso! Voltarei sempre 😘'

# Obter Comentarios


In [ ]:
# listagens de comentarios
comments_blob = []

#Obter lista dos comentários
comentarios = data.reviewBody

# Iteração em cada comentário
for comentario in comentarios:
  # Obter sentenças tratando emojis
  aux = TextBlob(str(comentario).encode('utf-16', 'surrogatepass').decode('utf-16'))
  comments_blob.append(aux)

#Obter sentenças

In [ ]:
# criar uma lista para  salvar sentenças //salvar os blobs//
comentario_blob = []
sentences_pt = []

# Iteração em cada review
for aux in comments_blob:
  for sentence in aux.sentences:
    # elimina sentenças irrelevantes
    if sentence != "!" and sentence != "." and sentence != ".." and sentence != "rs" and sentence != "Aff." :
      comentario_blob.append(sentence)
      sentences_pt.append(sentence)

comentario_blob[:10]


[Sentence("Comida maravilhosa, lugar do verdadeiro chiclete de camarão."),
 Sentence("O garçom Éric Lima nos atendeu muito bem, pretendemos voltar assim que possível."),
 Sentence("Comida boa, drinks legais e um ambiente bem aconchegante."),
 Sentence("Provamos muitas coisas e tudo nos agradou."),
 Sentence("Principalmente um pato servido em um dos pratos da casa, sensacional!"),
 Sentence("Excelente pra ir com a família!"),
 Sentence("Ótima Comida, ambiente agradável, bom atendimento e estacionamento."),
 Sentence("Vale a pena conhecer"),
 Sentence("Tomo sorvete na Visabor faz mais de 30 anos."),
 Sentence("Sabores de frutas ou artificiais.")]

#Salvar no Drive

Montar drive

In [ ]:
drive.mount('/content/drive/', force_remount='true')

Mounted at /content/drive/


Salvar no drive documento com comentários para tradução posterior. 

In [ ]:
!ls -la 'drive/My Drive/sentiment_analysis_project/'

drive_file = "drive/My Drive/sentiment_analysis_project/translate.csv" 
file = open(drive_file,'r')

data_translate = pd.read_csv(file)

# Iteração em cada review
for cmt in comentarios:
  try:
    #Obter sentenças tratando emojis
    comment_aux = str(cmt).encode('utf-16', 'surrogatepass').decode('utf-16')
    #cria objeto para adicionar ao dataframe
    obj_aux = { 'review': comment_aux, 'review_us': 'EMPTY' }
    # adiciona ao dataframe
    data_translate = data_translate.append(obj_aux, ignore_index=True)
  except:
    print("exception: " + cmt)

# Salva arquivo no google Drive
data_translate.to_csv(drive_file, index=False, header=True)    
data_translate

total 1
-rw------- 1 root root 18 Nov 11 01:16 translate.csv


,review,review_us
0,"Comida maravilhosa, lugar do verdadeiro chicle...",EMPTY
1,"Comida boa, drinks legais e um ambiente bem ac...",EMPTY
2,"Excelente pra ir com a família! Ótima Comida, ...",EMPTY
3,Tomo sorvete na Visabor faz mais de 30 anos. S...,EMPTY
4,O melhor hamburger que já comi! Excelente aten...,EMPTY
...,...,...
99995,"Ainda não inaugurado oficialmente, está atende...",EMPTY
99996,"Bom atendimento, lugar bonito, comida muito go...",EMPTY
99997,O Rio Scenarium é uma das baladas mais famosas...,EMPTY
99998,"Fui por indicação e com boa expectativa!\n116,...",EMPTY


#Traduzir comentario //sentenças//

Comentários traduzidos para Inglês (US) a partir de CSV no google drive.

In [ ]:
drive_file = "drive/My Drive/sentiment_analysis_project/translate_us.csv" 
data_translate = pd.read_csv(drive_file)

dt = data_translate.loc[data_translate['review_us'] == 'EMPTY']

#Informação sobre quantos faltam
print(dt.describe())
#Obter lista dos comentários
comentarios = dt.review

# Iteração em cada review. Colocar o número de interações que deseja realizar.
# QTDE   | Minutos
# 500    | 4,5
# 1000   | 8,9
# 1500   | 13,51
# 3000   | 26,86
# 5000   | 44,73
for comentario in comentarios[:50]:
  try:
    time.sleep(0.5)
    # Obter sentenças tratando emojis
    txt_blob_aux = TextBlob(str(comentario).encode('utf-16', 'surrogatepass').decode('utf-16'))
    aux_us = str(txt_blob_aux.translate(to='us'))
    print(aux_us)

    # Atualiza dataframe
    data_translate.at[(data_translate['review'] == comentario), 'review_us'] = aux_us
  except:
    print("EXCEPTION: " + str(txt_blob_aux))
    if txt_blob_aux.detect_language() == 'en':
      # Atualiza dataframe
      data_translate.at[(data_translate['review'] == comentario), 'review_us'] = str(txt_blob_aux)
      print("Up-to-date: " + str(txt_blob_aux))
      


# Salva arquivo no google Drive
data_translate.to_csv(drive_file, index=False, header=True)

data_translate

FileNotFoundError: ignored

##Sugestão de Refactory

In [ ]:
# criar uma lista de comentarios traduzidos
comments_blob_us = []

for txt_blob in comments_blob[:10]:
  try:
    blob_us = txt_blob.translate(to='us')
    comments_blob_us.append(blob_us)
    time.sleep(1)
  except:
    print('erro: ' + txt_blob)
    
comments_blob_us

#Gerar Senteças aleatórias

In [ ]:
# comentários com tradução
drive_file = "drive/My Drive/sentiment_analysis_project/translate_us.csv" 
data_random = pd.read_csv(drive_file)
# gerar novo dataset
data_sentencas = pd.DataFrame(columns= ['review', 'subjetividade', 'classificacao'])
# Tamanho do dataset de comentários em pt_br e us
size = len(data_random)
# auxiliares loop
max_sentences = 200
current = 0
array_pos = []

while current < max_sentences:
  i = random.randint(0, size)
  # Valida se a posição randômica já foi utilizada
  if i not in array_pos:
    #Obter sentenças tratando emojis
    comment_aux = data_random.iloc[i]
    txtb_review = TextBlob(str(comment_aux.review).encode('utf-16', 'surrogatepass').decode('utf-16'))
    # obter uma sentença aleatória do review sorteado
    sentence_br = random.choice(txtb_review.sentences)
    print('REVIEW: '+ str(txtb_review))
    print('SENTENÇA: '+ str(sentence_br))

    #cria objeto para adicionar ao dataframe
    if sentence_br != \"!\" and sentence_br != \".\" and sentence_br != \"..\" and sentence_br != \"rs\" and sentence_br != \"Aff.\" :
      obj_aux = { 'review': sentence_br, 'subjetividade': '', 'classificacao': '' }
      # adiciona ao dataframe
      data_sentencas = data_sentencas.append(obj_aux, ignore_index=True)
      current += 1
    
  array_pos.append(i)
  print(array_pos)

# Salvar no drive
data_sentencas.to_csv('drive/My Drive/sentiment_analysis_project/ground_truth.csv', index=False)
data_sentencas



O lugar é despretensioso, mas limpo, e cada vez que vou ao local tem um tipo de cachorro quente diferente, atraindo cada vez mais clientela.
Os combos do lanche de 22 cm, mais refrigerante e um churros (de doce de leite ou chocolate) são super em conta e montam uma refeição completa!
[46782]
Já estive em vários restaurantes de comida japonesa, mas até hoje nenhum se comparou ao Kawa da Alameda Lorena.
Os pratos são deliciosos e diferentes e o atendimento é ótimo.
Além disso, o Kawa da Lorena é um restaurante conceito e a decoração é moderna e sofisticada.
Recomendo!
[46782, 19187]
Excelentes hambúrgueres que são montados conforme a escolha de ingredientes feita pelo cliente.
Carta de cervejas importadas e nacionais muito bem sortida.
Atendimento atencioso e cortês.
Ambiente agradável.
Preço justo.
Ótima opção para refeição informal com amigos ou família.
[46782, 19187, 49839]
Restaurante francês de muito boa qualidade.
Necessita reserva previa.
Vale a pena conferir.
Cozinha com o chef 

,review,review_us,subjetividade
0,"(O, , l, u, g, a, r, , é, , d, e, s, p, r, ...","(T, h, e, , p, l, a, c, e, , i, s, , u, n, ...",
1,"(O, s, , c, o, m, b, o, s, , d, o, , l, a, ...","(T, h, e, , 2, 2, , c, m, , s, n, a, c, k, ...",
2,"(J, á, , e, s, t, i, v, e, , e, m, , v, á, ...","(I, , h, a, v, e, , b, e, e, n, , t, o, , ...",
3,"(O, s, , p, r, a, t, o, s, , s, ã, o, , d, ...","(T, h, e, , d, i, s, h, e, s, , a, r, e, , ...",
4,"(A, l, é, m, , d, i, s, s, o, ,, , o, , K, ...","(I, n, , a, d, d, i, t, i, o, n, ,, , K, a, ...",
...,...,...,...
195,"(F, r, e, q, u, e, n, t, a, d, o, s, , p, o, ...","(F, r, e, q, u, e, n, t, e, d, , b, y, , n, ...",
196,"(D, i, v, e, r, s, ã, o, , g, a, r, a, n, t, ...","(G, u, a, r, a, n, t, e, e, d, , f, u, n, , ...",
197,"(O, s, , s, a, n, d, u, í, c, h, e, s, , s, ...","(T, h, e, , s, a, n, d, w, i, c, h, e, s, , ...",
198,"(E, s, t, i, v, e, , c, o, m, , a, m, i, g, ...","(I, , w, a, s, , w, i, t, h, , f, r, i, e, ...",


##Sugestão de Refactory

In [1]:
import random

# gera 200 index aleatorios não repetidos
indexRandom = []

while len(indexRandom) < 200:
  n = random.randint(0, len(comments_blob) -1)
  if not n in indexRandom:
    indexRandom.append(n)

len(indexRandom)

NameError: ignored

In [ ]:
list_sentences = [];

for i in indexRandom:
  if len(list_sentences) == 200:
      break
  # Descomentar somente se tiver sido feito a traducao no 'comments_blob_us'
  # for sentence_br, sentence in zip(comments_blob_limite[i].sentences, comments_blob_us[i].sentences):
  for sentence_br in comments_blob[i].sentences:
    newSentence = {
        "sentence_pt": str(sentence_br).encode('utf-16', 'surrogatepass').decode('utf-16'),
        # "sentence_us": str(sentence).encode('utf-16', 'surrogatepass').decode('utf-16'),
        # "subjectivity": sentence.sentiment.subjectivity,
        # "subjectivity": sentence.sentiment.polarity,
        # "index_comment": i
    }
    list_sentences.append(newSentence)
    if len(list_sentences) == 200:
      break

len(list_sentences)

In [ ]:
#datasete sentences
dtSentence = pd.DataFrame(list_sentences)

dtSentence.head()

dtSentence.to_csv(r'sentences.csv')

# Ground Truth Comparativo



In [5]:

fonte = "https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/ground_truth.csv?raw=true"

dt_ground = pd.read_csv(fonte)
dt_ground.columns = ['comments_pt', 'subjectivity', 'polarity']
dt_ground.head()

,comments_pt,subjectivity,polarity
0,"comida excelente, atendimento bom é perto do c...",NaN,NaN
1,Parabens Camarada!,NaN,NaN
2,"Restaurante fantástico, ambiente aconchegante,...",NaN,NaN
3,O restaurante possui um espaço kids muito segu...,NaN,NaN
4,O preço cobrado é justo.,NaN,NaN


# Cálculo da subjetividade

In [ ]:
# coloquei para teste, uso as sentenças dos dez primeiros comentarios
# O valor da subjetividade é um número real entre [0.0, 1.0], onde 0.0 o texto é indicado como muito objetivo e 1.0 o texto é indicado como muito subjetivo.
for i in range(10):
  for sentence_br, sentence in zip(comments_blob_limite[i].sentences, comments_blob_us[i].sentences):
    print(sentence_br, ':', sentence.sentiment.subjectivity)

# Cálculo da polaridade

In [ ]:
# coloquei para teste, uso as sentenças dos dez primeiros comentarios
# O valor da polaridade é um número real entre os valores [-1.0, 1.0].
for i in range(10):
  for sentence_br, sentence in zip(comments_blob_limite[i].sentences, comments_blob_us[i].sentences):
    print(sentence_br, ':', sentence.sentiment.polarity)

Obter dataset traduzido com textblob e gerar sentenças aleatórias para o ***ground-truth***/